<a href="https://colab.research.google.com/github/sonjoy2074/Twitter-Sentiment-Analysis-NLP/blob/main/Twitter_Sentiment_Analysis_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
#API to fetch the dataset
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.22GB/s]


In [4]:
#extracting the compressed dataset

from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("The dataset is extracted")

The dataset is extracted


In [5]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [8]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [9]:
#checking gthe number of rows and colums
twitter_data.shape

(1599999, 6)

In [10]:
#printing the first 5 of the dataframe
twitter_data.head()

0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY _TheSpecialOne_  \
0  0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   scotthamilton   
1  0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY        mattycus   
2  0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY         ElleCTF   
3  0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY          Karoli   
4  0  1467811372  Mon Apr 06 22:20:00 PDT 2009  NO_QUERY        joy_wolf   

  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  
0  is upset that he can't update his Facebook by ...                                                                   
1  @Kenichan I dived many times for the ball. Man...                                                                   
2    my whole body feels itchy and like its on fire                                                                    
3  @nationwideclass no, it's not behaving at all....                                                                   
4                      @Kwesidei not the whole crew

In [11]:
colum_name=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=colum_name, encoding='ISO-8859-1')

In [12]:
twitter_data.shape

(1600000, 6)

In [13]:
twitter_data.head()

target          id                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....

In [14]:
#counting the number of missing value in the dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [15]:
# checking the distribution fo target colum
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

Convert the targate "4" to "1"

In [16]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [17]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0--->Negative Twitte
2--->Positive Twitte

In [18]:
port_stem= PorterStemmer()

In [21]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content
                   if not word in stopwords.words('english')
                   ]
  stemmed_content=''.join(stemmed_content)
  return stemmed_content

In [22]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [23]:
twitter_data.head()

target          id                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  \
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1    scotthamilton  is upset that he can't update his Facebook by ...   
2         mattycus  @Kenichan I dived many times for the ball. Man...   
3          ElleCTF    my whole body feels itchy and like its on fire    
4           Karoli  @nationwideclass no, it's not behaving at all....   

                                     stemmed_content  
0  switchfoothttptwitpiccomzlawwwbummershouldagot...  
1  upsetupdatfacebooktextmightcriresultschooltoda...  
2       kenichandivemanitimeballmanagsaverestgobound  
3                         wholebodifeelitchilikefire  
4                         nationwideclassbehavmadsee

In [24]:
print(twitter_data['stemmed_content'])

0          switchfoothttptwitpiccomzlawwwbummershouldagot...
1          upsetupdatfacebooktextmightcriresultschooltoda...
2               kenichandivemanitimeballmanagsaverestgobound
3                                 wholebodifeelitchilikefire
4                                 nationwideclassbehavmadsee
                                 ...                        
1599995                               wokeschoolbestfeelever
1599996      thewdbcomcoolhearoldwaltinterviewhttpblipfmbmta
1599997                             readimojomakeovaskdetail
1599998           happithbirthdaybooallltimetupacamarushakur
1599999    happicharitytuesdaythenspccsparkscharspeakinguphh
Name: stemmed_content, Length: 1600000, dtype: object


In [25]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [53]:
X=twitter_data['stemmed_content']
Y=twitter_data['target'].values

In [54]:
print(Y)

[0 0 0 ... 1 1 1]


In [55]:
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [56]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [38]:
print(X_test)

131348     mmangenfinemuchtimechattwitterhubbibacksummera...
1142114                 ahmayshowwruthkimampgeoffreysanhueza
244564                        ishataramaybbayareathangdammit
445353                                    gameendloststinkyy
415893                                           coolbrother
                                 ...                        
178459     twitterdrivenutwontletdownloadprofilpicguesske...
1515130                                     teamqivanawelcom
1449952        destinineverthelesshooraymemberwondersafetrip
441063                                              feelwell
1583304                                     supersandrothank
Name: stemmed_content, Length: 320000, dtype: object


In [57]:
vectorize=TfidfVectorizer()

X_train=vectorize.fit_transform(X_train)
X_test=vectorize.transform(X_test)

In [58]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1279572 stored elements and shape (1280000, 1239526)>
  Coords	Values
  (0, 1163354)	1.0
  (1, 435884)	1.0
  (2, 283352)	1.0
  (3, 1062770)	1.0
  (4, 1091794)	1.0
  (5, 197786)	1.0
  (6, 577996)	1.0
  (7, 114435)	1.0
  (8, 782552)	1.0
  (9, 482000)	1.0
  (10, 280992)	1.0
  (11, 898312)	1.0
  (12, 1137048)	1.0
  (13, 386454)	1.0
  (14, 992231)	1.0
  (15, 1236032)	1.0
  (16, 23878)	1.0
  (17, 330941)	1.0
  (18, 433866)	1.0
  (19, 175299)	1.0
  (20, 890756)	1.0
  (21, 456873)	1.0
  (22, 327755)	1.0
  (23, 219744)	1.0
  (24, 1238202)	1.0
  :	:
  (1279975, 96044)	1.0
  (1279976, 1074177)	1.0
  (1279977, 890012)	1.0
  (1279978, 949326)	1.0
  (1279979, 744769)	1.0
  (1279980, 800670)	1.0
  (1279981, 530027)	1.0
  (1279982, 905736)	1.0
  (1279983, 480531)	1.0
  (1279984, 72766)	1.0
  (1279985, 57198)	1.0
  (1279986, 979289)	1.0
  (1279987, 728143)	1.0
  (1279988, 1198114)	1.0
  (1279989, 436984)	1.0
  (1279990, 840415)	1.0
  (127999

Training the Machine Learning Model

Logistic Regression

In [63]:
model= LogisticRegression(max_iter=1000)

In [64]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Accurecy score

In [65]:
X_train_prediction=model.predict(X_train)
training_data_accurecy=accuracy_score(Y_train, X_train_prediction)

In [66]:
print("Accurecy socre on the training data: ",training_data_accurecy)

Accurecy socre on the training data:  0.99614140625


In [67]:
X_test_prediction=model.predict(X_test)
test_data_accurecy=accuracy_score(Y_test,X_test_prediction)

In [68]:
print("Accurecy socre on the training data: ",test_data_accurecy)

Accurecy socre on the training data:  0.519303125
